In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import random
from matplotlib import pyplot as plt

In [2]:
def canny_img(img):
    # 第1步：完成一阶有限差分计算，计算每一点的梯度幅值与方向
    img1 = np.zeros(img.shape,dtype="uint8") # 与原图大小相同
    theta = np.zeros(img.shape,dtype="float")  # 方向矩阵原图像大小
    img = cv2.copyMakeBorder(img,1,1,1,1,borderType=cv2.BORDER_REPLICATE)
    rows,cols = img.shape
    for i in range(1,rows-1):
        for j in range(1,cols-1):
            # Gy
            Gy = (np.dot(np.array([1, 1, 1]), (m1 * img[i - 1:i + 2, j - 1:j + 2]))).dot(np.array([[1], [1], [1]]))
            # Gx
            Gx = (np.dot(np.array([1, 1, 1]), (m2 * img[i - 1:i + 2, j - 1:j + 2]))).dot(np.array([[1], [1], [1]]))
            if Gx[0] == 0:
                theta[i-1,j-1] = 90
                continue
            else:
                temp = (np.arctan(Gy[0] / Gx[0]) ) * 180 / np.pi
            if Gx[0]*Gy[0] > 0:
                if Gx[0] > 0:
                    theta[i-1,j-1] = np.abs(temp)
                else:
                    theta[i-1,j-1] = (np.abs(temp) - 180)
            if Gx[0] * Gy[0] < 0:
                if Gx[0] > 0:
                    theta[i-1,j-1] = (-1) * np.abs(temp)
                else:
                    theta[i-1,j-1] = 180 - np.abs(temp)
            img1[i-1,j-1] = (np.sqrt(Gx**2 + Gy**2))
    for i in range(1,rows - 2):
        for j in range(1, cols - 2):
            if (((theta[i,j] >= -22.5) and (theta[i,j]< 22.5) ) or
                    ( (theta[i,j] <= -157.5) and (theta[i,j] >= -180) ) or
                    ( (theta[i,j] >= 157.5) and (theta[i,j] < 180) ) ):
                theta[i,j] = 0.0
            elif (    ( (theta[i,j] >= 22.5) and (theta[i,j]< 67.5) ) or
                      ( (theta[i,j] <= -112.5) and (theta[i,j] >= -157.5) ) ):
                theta[i,j] = 45.0
            elif (    ( (theta[i,j] >= 67.5) and (theta[i,j]< 112.5) ) or
                      ( (theta[i,j] <= -67.5) and (theta[i,j] >= -112.5) ) ):
                theta[i,j] = 90.0
            elif (    ( (theta[i,j] >= 112.5) and (theta[i,j]< 157.5) ) or
                      ( (theta[i,j] <= -22.5) and (theta[i,j] >= -67.5) ) ):
                theta[i,j] = -45.0

    # 第2步：进行 非极大值抑制计算
    img2 = np.zeros(img1.shape) # 非极大值抑制图像矩阵

    for i in range(1,img2.shape[0]-1):
        for j in range(1,img2.shape[1]-1):
            if (theta[i,j] == 0.0) and (img1[i,j] == np.max([img1[i,j],img1[i+1,j],img1[i-1,j]]) ):
                img2[i,j] = img1[i,j]

            if (theta[i,j] == -45.0) and img1[i,j] == np.max([img1[i,j],img1[i-1,j-1],img1[i+1,j+1]]):
                img2[i,j] = img1[i,j]

            if (theta[i,j] == 90.0) and  img1[i,j] == np.max([img1[i,j],img1[i,j+1],img1[i,j-1]]):
                img2[i,j] = img1[i,j]

            if (theta[i,j] == 45.0) and img1[i,j] == np.max([img1[i,j],img1[i-1,j+1],img1[i+1,j-1]]):
                img2[i,j] = img1[i,j]

    # 第3步：双阈值检测和边缘连接
    img3 = np.zeros(img2.shape) #定义双阈值图像
    TL = 0.4*np.max(img2)
    TH = 0.5*np.max(img2)
    # TL = 50
    # TH = 100
    #关键在这两个阈值的选择
    for i in range(1,img3.shape[0]-1):
        for j in range(1,img3.shape[1]-1):
            if img2[i,j] < TL:
                img3[i,j] = 0
            elif img2[i,j] > TH:
                img3[i,j] = 255
            elif (( img2[i+1,j] < TH) or (img2[i-1,j] < TH )or( img2[i,j+1] < TH )or
                  (img2[i,j-1] < TH) or (img2[i-1, j-1] < TH )or ( img2[i-1, j+1] < TH) or
                  ( img2[i+1, j+1] < TH ) or ( img2[i+1, j-1] < TH) ):
                img3[i,j] = 255

    return img3


    # cv2.imshow("1",img)  		  # 原始图像
    # cv2.imshow("2",img1)       # 梯度幅值图
    # cv2.imshow("3",img2)       #非极大值抑制灰度图
    # cv2.imshow("4",img3)       # 最终效果图
    # cv2.imshow("theta",theta) #角度值灰度图
    # cv2.waitKey(0)


In [3]:
root  = r'F:\CRC&STAD_image\image_class_sample'
root3 = r'E:\Graduation_design'
os.chdir(root3)
m1 = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
m2 = np.array([[-1,-1,-1],[0,0,0],[1,1,1]])
data = pd.DataFrame()
for file in os.listdir(root):
        if os.listdir(root+os.sep+file):
            tumor_png = random.choice(os.listdir(root+os.sep+file))
            img = cv2.imread(root+os.sep+file+os.sep+tumor_png,0)  #原始图片像素矩阵224*224
            img = cv2.GaussianBlur(img,(3,3),2)
            matrix = canny_img(img)  #提取出来的canny矩阵是224*224
            matrix_mean = matrix.mean(0)   #按行平均 224*1
            data[file] = list(matrix_mean)


In [4]:
data

,TCGA.3L.AA1B.01A,TCGA.3M.AB46.01A,TCGA.4N.A93T.01A,TCGA.5M.AAT4.01A,TCGA.5M.AAT6.01A,TCGA.5M.AATE.01A,TCGA.A6.2671.01A,TCGA.A6.2681.01A,TCGA.A6.2685.01A,TCGA.A6.2686.01A,...,TCGA.VQ.AA68.01A,TCGA.VQ.AA69.01A,TCGA.VQ.AA6A.01A,TCGA.VQ.AA6D.01A,TCGA.VQ.AA6F.01A,TCGA.VQ.AA6G.01A,TCGA.VQ.AA6J.01A,TCGA.VQ.AA6K.01A,TCGA.WS.AB45.01A,TCGA.ZA.A8F6.01A
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,5.691964,1.138393,28.459821,45.535714,10.245536,13.660714,2.276786,6.830357,11.383929,...,18.214286,23.906250,25.044643,28.459821,23.906250,27.321429,27.321429,17.075893,25.044643,9.107143
2,0.000000,21.629464,28.459821,25.044643,50.089286,25.044643,13.660714,10.245536,9.107143,11.383929,...,40.982143,36.428571,26.183036,18.214286,27.321429,19.352679,19.352679,21.629464,34.151786,12.522321
3,0.000000,12.522321,10.245536,22.767857,46.674107,9.107143,10.245536,11.383929,11.383929,6.830357,...,22.767857,20.491071,37.566964,29.598214,25.044643,19.352679,31.875000,19.352679,18.214286,5.691964
4,0.000000,15.937500,15.937500,28.459821,38.705357,17.075893,11.383929,13.660714,6.830357,6.830357,...,38.705357,26.183036,21.629464,28.459821,19.352679,34.151786,43.258929,17.075893,34.151786,2.276786
5,0.000000,25.044643,15.937500,23.906250,50.089286,20.491071,13.660714,12.522321,11.383929,12.522321,...,31.875000,27.321429,40.982143,29.598214,45.535714,40.982143,25.044643,29.598214,17.075893,2.276786
6,0.000000,17.075893,15.937500,27.321429,36.428571,11.383929,10.245536,6.830357,6.830357,17.075893,...,26.183036,23.906250,17.075893,54.642857,12.522321,42.120536,37.566964,27.321429,26.183036,2.276786
7,0.000000,19.352679,39.843750,27.321429,66.026786,11.383929,14.799107,6.830357,15.937500,10.245536,...,40.982143,28.459821,31.875000,36.428571,14.799107,34.151786,30.736607,34.151786,29.598214,1.138393
8,0.000000,7.968750,31.875000,21.629464,56.919643,22.767857,10.245536,6.830357,7.968750,27.321429,...,31.875000,13.660714,43.258929,37.566964,22.767857,46.674107,58.058036,30.736607,10.245536,2.276786
9,0.000000,10.245536,31.875000,29.598214,64.888393,11.383929,4.553571,12.522321,19.352679,19.352679,...,40.982143,43.258929,23.906250,38.705357,28.459821,38.705357,39.843750,29.598214,14.799107,2.276786


In [5]:
root3 = r'E:\Graduation_design'
os.chdir(root3)
data.to_csv('pig_dimna.csv')

0


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\wang0\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\wang0\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-18-e6065d3c383a>", line 10, in timer
    time.sleep(1)
NameError: name 'time' is not defined



0


deque([1, 2, 3, 4, 5])
deque([2, 3, 4, 5])


In [27]:
df1 = pd.DataFrame([[1,2,3,4], [6,7,8,9]], columns=['D', 'B', 'E', 'A'], index=[1,2])

df2 = pd.DataFrame([[10,20,30,40], [60,70,80,90], [600,700,800,900]], columns=['A', 'B', 'C', 'D'], index=[1,2,3])

In [28]:
print(df1)
print(df2)

   D  B  E  A
1  1  2  3  4
2  6  7  8  9
     A    B    C    D
1   10   20   30   40
2   60   70   80   90
3  600  700  800  900


In [29]:
df1,df2= df1.align(df2,join = 'inner')

In [30]:
print(df1)
print(df2)

   D  B  A
1  1  2  4
2  6  7  9
    D   B   A
1  40  20  10
2  90  70  60


In [31]:
def mul(x):
    return x*x
a = [1,2,3,4,5]
er = list(map(mul,a))
print(er)

[1, 4, 9, 16, 25]


In [3]:
import numpy as np
s = np.array([[1,2,3],[3,4,5]])
flatten(s)

NameError: name 'flatten' is not defined